In [ ]:
dataset_real = np.loadtxt("/kaggle/input/train-test-amazon1/train_test_amazon1.sample", delimiter=' ', dtype=int)
y_real = dataset_real[:, -1:]
amazon_PCA_real = np.loadtxt('/kaggle/input/amazon-kaggle-with-pca/Amazon_Kaggle_with_PCA.csv', delimiter=',', dtype=np.float64, skiprows=1)
X_real = amazon_PCA_real[:, 1:]
X_train_real, X_test_real, y_train_real, y_test_real = train_test_split(X_real, y_real, test_size=0.2, random_state=42)

# Reshape data to have three dimensions (height, width, and channels)
X_train_image_real = X_train_real.reshape((-1, X_train_real.shape[1], 1))
X_test_image_real = X_test_real.reshape((-1, X_test_real.shape[1], 1))

# Determine batch size
batch_size_real = 64

# Create the input layer
inputs_real = Input(shape=(X_train_image_real.shape[1], X_train_image_real.shape[2], 1))

# Inception-like module
def inception_module(x, filters):
    conv1x1 = Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)
    conv3x3 = Conv2D(filters[1], (3, 3), padding='same', activation='relu')(x)
    conv5x5 = Conv2D(filters[2], (5, 5), padding='same', activation='relu')(x)
    maxpool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    maxpool_conv = Conv2D(filters[3], (1, 1), padding='same', activation='relu')(maxpool)

    inception_block = concatenate([conv1x1, conv3x3, conv5x5, maxpool_conv], axis=-1)
    return inception_block

# Build the Inception model
x_real = inception_module(inputs_real, filters=[64, 128, 128, 64])
x_real = inception_module(x_real, filters=[128, 192, 192, 128])
x_real = inception_module(x_real, filters=[192, 256, 256, 192])

# Global Average Pooling
x_real = GlobalAveragePooling2D()(x_real)

# Fully Connected Layer
output_real = Dense(1, activation='sigmoid', name='output_real')(x_real)  # Assuming you have 4 classes

# Create the model
model_real = Model(inputs_real, output_real)
optimizer_real = Adam(learning_rate=0.0001)
# Compile the model with an appropriate optimizer, loss function, and metrics
model_real.compile(optimizer=optimizer_real, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_real.fit(
    X_train_image_real,
    y_train_real,
    epochs=30,
    batch_size=batch_size_real,
    validation_data=(X_test_image_real, y_test_real)
)
scores_real = model_real.evaluate(X_test_image_real, y_test_real)
np.savetxt('scores_real.txt', scores_real, delimiter=',', fmt='%f')
print("scores:", scores_real)
y_pred_probs_real = model_real.predict(X_test_image_real)
np.savetxt('y_pred_probs_real.txt', y_pred_probs_real, delimiter=',', fmt='%f')
print("y_pred_probs:", y_pred_probs_real)
y_pred_real = np.round(y_pred_probs_real)
np.savetxt('y_pred_real.txt', y_pred_real, delimiter=',', fmt='%f')
print("y_pred:", y_pred_real)
fpr_real, tpr_real, _ = roc_curve(y_test_real.ravel(), y_pred_probs_real.ravel())
print("fpr:", fpr_real)
np.savetxt('fpr_real.txt', fpr_real, delimiter=',', fmt='%f')
print("tpr:", tpr_real)
np.savetxt('tpr_real.txt', tpr_real, delimiter=',', fmt='%f')
roc_auc_real = roc_auc_score(y_test_real.ravel(), y_pred_probs_real.ravel())
print("roc_auc:", roc_auc_real)
precision_real, recall_real, _ = precision_recall_curve(y_test_real.ravel(), y_pred_probs_real.ravel())
print("precision:", precision_real)
print("recall:", recall_real)
np.savetxt('precision_real.txt', precision_real, delimiter=',', fmt='%f')
np.savetxt('recall_real.txt', recall_real, delimiter=',', fmt='%f')
average_precision_real = average_precision_score(y_test_real.ravel(), y_pred_probs_real.ravel())

cm_real = confusion_matrix(y_test_real.ravel(), y_pred_real.ravel())
f1_real = f1_score(y_test_real.ravel(), y_pred_real.ravel())
tpr_value_real = cm_real[1, 1] / (cm_real[1, 1] + cm_real[1, 0])
fpr_value_real = cm_real[0, 1] / (cm_real[0, 1] + cm_real[0, 0])

print("ROC AUC:", roc_auc_real)
print("Precision-Recall AUC:", average_precision_real)
print("Confusion Matrix:")
print(cm_real)
print("F1-Score:", f1_real)
print("True Positive Rate (TPR):", tpr_value_real)
print("False Positive Rate (FPR):", fpr_value_real)

# Plot ROC curve
plt.figure()
plt.plot(fpr_real, tpr_real, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc_real)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.savefig('/kaggle/working/Inception_Real_PCA_roc_curve_real.png')
plt.show()

# Plot Precision-Recall curve
plt.figure()
plt.plot(recall_real, precision_real, color='blue', lw=2, label='Precision-Recall curve (AUC = %0.2f)' % average_precision_real)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.savefig('/kaggle/working/Inception_Real_PCA_precision_recall_curve_real.png')
plt.show()

with open('/kaggle/working/Inception_real_PCA_results.txt', 'w') as file:
    file.write(f"ROC AUC: {roc_auc_real}\n")
    file.write(f"Precision-Recall AUC: {average_precision_real}\n")
    file.write(f"Confusion Matrix:\n{cm_real}\n")
    file.write(f"F1-Score: {f1_real}\n")
    file.write(f"True Positive Rate (TPR): {tpr_value_real}\n")
    file.write(f"False Positive Rate (FPR): {fpr_value_real}\n")
    file.write(f"tpr: {tpr_real}\n")
    file.write(f"fpr: {fpr_real}\n")
    file.write(f"precision: {precision_real}\n")
    file.write(f"recall: {recall_real}\n")
    file.write(f"y_pred: {y_pred_real}\n")

In [ ]:
import json

# Assuming tpr and fpr are NumPy arrays
print("Type of f1:", type(f1))
print("Type of precision:", type(precision))
print("Type of recall:", type(recall))
print("Type of tpr_value:", type(tpr_value))
print("Type of fpr_value:", type(fpr_value))
print("Type of roc_auc:", type(roc_auc))
print("Type of prc_auc:", type(average_precision))
print("Type of tpr:", type(tpr))
print("Type of fpr:", type(fpr))

precision_list = precision.tolist()
recall_list = recall.tolist()
tpr_list = tpr.tolist()
fpr_list = fpr.tolist()

#print(f1.type())

data = {
    "Average F1 Score": float(f1), 
    #"Average Precision": precision, 
    #"Average Recall": float(recall), 
    "Average True Positive Rate": float(tpr_value), 
    "Average False Positive Rate": float(fpr_value), 
    "Average ROC AUC": roc_auc, 
    "Average PRC AUC": average_precision,
    "TPR Array" : tpr_list,
    "FPR Array" : fpr_list,
    "Precision Array": precision_list, 
    "Recall Array": recall_list
}

# Specify the file path
file_path = "/kaggle/working/Inception - Real PCA.json"

# Write the data to a JSON file
with open(file_path, "w") as json_file:
    json.dump(data, json_file)